# Document Ingestion with LiteIngestion 📚

This notebook demonstrates how to use the `LiteIngestion` class from `lite_agents` to perform optimized ingestion of company policy documents into a ChromaDB database.

## Ingestion Pipeline ⚙️

The `LiteIngestion` class manages the entire RAG pipeline:
1.  **Reading**: Loads files from the specified directory.
2.  **Chunking**: Splits text into manageable chunks.
3.  **Embedding**: Generates vector embeddings for each chunk.
4.  **Storage**: Saves chunks and embeddings to the Vector DB (ChromaDB).

## Contextual Retrieval (Optional) 🧠

Contextual Retrieval (inspired by Anthropic) improves retrieval quality by adding context to each chunk before embedding. It answers:
- Which document does this chunk belong to?
- What is the main topic of this chunk?
- How does it relate to the overall document context?

You can enable this feature by setting `add_context=True` in `LiteIngestion`.

## 1. Imports and Setup 🛠️

In [5]:
from pathlib import Path
from typing import List
import json
import time
from lite_agents.llm import LiteLLM
from lite_agents.db import ChromaDB
from lite_agents.ingestion import LiteIngestion
from litellm import embedding
from tqdm.auto import tqdm

# Configuration
DATA_DIR = Path("../data")
POLICIES_DIR = DATA_DIR / "company_policies"
CHROMA_PATH = DATA_DIR / "chroma_db"
COLLECTION_NAME = "company_policies"
CHUNK_SIZE = 800  # characters per chunk
CHUNK_OVERLAP = 200  # overlap between chunks
EMBEDDING_MODEL = "text-embedding-3-small"  # OpenAI embedding model
LLM_MODEL = "gpt-5-nano-2025-08-07"  # Model for contextual retrieval
ADD_CONTEXT = True  # Enable contextual retrieval

In [2]:
def create_embeddings_batch(texts: List[str]) -> List[List[float]]:
    """Creates embeddings using litellm.
    
    Args:
        texts (List[str]): list of texts to embed
        
    Returns:
        List[List[float]]: list of embedding vectors
    """
    embeddings = []
    
    # Batch of 100 to avoid rate limits
    batch_size = 100
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
        batch = texts[i:i+batch_size]
        response = embedding(model=EMBEDDING_MODEL, input=batch)
        batch_embeddings = [item['embedding'] for item in response.data]
        embeddings.extend(batch_embeddings)
    
    return embeddings

## 2. LiteIngestion Initialization 🚀

We create an instance of the class that will handle the entire process.

In [3]:
# Initialize LiteLLM
print("🤖 Initializing LiteLLM...")
llm = LiteLLM(model=LLM_MODEL)
print(f"✅ LiteLLM initialized with model: {LLM_MODEL}\n")

# Initialize ChromaDB
print("💾 Initializing ChromaDB...")
db = ChromaDB(
    collection_name=COLLECTION_NAME,
    path=CHROMA_PATH,
    persistent=True
)
print(f"✅ ChromaDB initialized: {CHROMA_PATH}/{COLLECTION_NAME}\n")

# Create LiteIngestion instance
print(f"🚀 Initializing LiteIngestion (add_context={ADD_CONTEXT})...")
ingestion = LiteIngestion(
    llm=llm,
    vector_db=db,
    embedding_function=create_embeddings_batch,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    add_context=ADD_CONTEXT
)
print("✅ LiteIngestion ready!\n")

🤖 Initializing LiteLLM...
✅ LiteLLM initialized with model: gpt-5-nano-2025-08-07

💾 Initializing ChromaDB...
✅ ChromaDB initialized: ../data/chroma_db/company_policies

🚀 Initializing LiteIngestion (add_context=True)...
✅ LiteIngestion ready!



In [4]:
t_start = time.time()
all_chunks = ingestion.process_directory(
    directory=POLICIES_DIR,
    file_pattern="*.md"
)
t_end = time.time()

print(f"\n✅ Processing complete! Total chunks: {len(all_chunks)}. Time taken: {t_end - t_start:.2f} seconds.\n")

INFO | __init__ | process_directory:248 | 📂 Found 5 files in ../data/company_policies
INFO | __init__ | process_document:161 | 📄 Processing document: POL-01-hr-ferie.md
INFO | __init__ | process_document:172 | 📝 Generating summary for: Gestione Ferie, Permessi e Assenze (POL-HR-001)
WARNING | __init__ | generate_document_summary:113 | ⚠️ Currently all the content is used for summary generation. Consider using a preview for large documents.
INFO | __init__ | generate_document_summary:121 | ⏱️ Document summary generated in 12.77 seconds.
INFO | __init__ | process_document:192 | ✂️ Created 13 chunks from POL-01-hr-ferie.md
INFO | __init__ | process_document:207 | ⏱️ Generated context for chunk 1/13 in 11.59 seconds.
INFO | __init__ | process_document:207 | ⏱️ Generated context for chunk 2/13 in 8.23 seconds.
INFO | __init__ | process_document:207 | ⏱️ Generated context for chunk 3/13 in 8.56 seconds.
INFO | __init__ | process_document:207 | ⏱️ Generated context for chunk 4/13 in 7.99 seco


✅ Processing complete! Total chunks: 52. Time taken: 537.17 seconds.



In [6]:
# Ingest chunks into ChromaDB
print("\n🔄 Ingesting into ChromaDB...\n")

ingestion.ingest_chunks(all_chunks)

print("\n🎉 INGESTION COMPLETED SUCCESSFULLY! 🎉")

INFO | __init__ | ingest_chunks:270 | 🚀 Starting ingestion of 52 chunks
INFO | __init__ | ingest_chunks:281 | 🧠 Creating embeddings...



🔄 Ingesting into ChromaDB...



Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.99s/it]
INFO | __init__ | ingest_chunks:285 | 💾 Adding to vector database...



🎉 INGESTION COMPLETED SUCCESSFULLY! 🎉


In [7]:
ingestion.save_chunks_to_json(
    chunks=all_chunks,
    output_path=DATA_DIR/"ingested_chunks.json"
)

INFO | __init__ | save_chunks_to_json:357 | 💾 Saving 52 chunks to ../data/ingested_chunks.json
INFO | __init__ | save_chunks_to_json:365 | ✅ Chunks saved successfully to ../data/ingested_chunks.json


In [8]:
# Show stats
stats = ingestion.get_statistics(all_chunks)
print(json.dumps(stats, indent=2, ensure_ascii=False))

{
  "total_documents": 5,
  "total_chunks": 52,
  "avg_chunks_per_document": 10.4,
  "documents": {
    "POL-01-hr-ferie": {
      "title": "Gestione Ferie, Permessi e Assenze (POL-HR-001)",
      "chunks": 13,
      "total_chars": 3132,
      "context_chars": 6302
    },
    "POL-02-lavoro-da-remoto": {
      "title": "Lavoro Agile e Smart Working (POL-HR-002)",
      "chunks": 10,
      "total_chars": 2607,
      "context_chars": 4763
    },
    "POL-03-spese-note-spese": {
      "title": "Gestione Spese e Rimborsi (POL-FIN-003)",
      "chunks": 10,
      "total_chars": 2835,
      "context_chars": 4813
    },
    "POL-04-viaggi-di-lavoro": {
      "title": "Viaggi e Trasferte (POL-FIN-004)",
      "chunks": 9,
      "total_chars": 2504,
      "context_chars": 4114
    },
    "POL-05-acquisti-fornitori": {
      "title": "Acquisti e Gestione Fornitori (POL-PROC-005)",
      "chunks": 10,
      "total_chars": 2534,
      "context_chars": 5313
    }
  }
}


## 3. Test Retrieval 🔎

We save ingestion information for future reference and test the retrieval quality.

In [9]:
def test_retrieval(db: ChromaDB, query: str, n_results: int = 3) -> None:
    """Tests retrieval with a query."""
    print(f"\n{'='*80}")
    print(f"🔍 QUERY: {query}")
    print(f"{'='*80}\n")
    
    # Create query embedding
    response = embedding(model=EMBEDDING_MODEL, input=[query])
    query_embedding = response.data[0]['embedding']
    
    # Execute query
    results = db.query(query_embeddings=query_embedding, n_results=n_results)
    
    if not results:
        print("❌ No results found.\n")
        return
    
    for i, result in enumerate(results, 1):
        print(f"\n{'─'*80}")
        print(f"RESULT #{i} (distance: {result['distance']:.4f})")
        print(f"{'─'*80}")
        print(f"📄 Document: {result['metadata']['document_title']}")
        print(f"📍 Chunk: {result['metadata']['chunk_index'] + 1}/{result['metadata']['total_chunks']}")
        if 'section_header' in result['metadata']:
            print(f"📑 Section: {result['metadata']['section_header']}")
        print(f"\n🔍 Context:")
        print(result['metadata']['context'])
        print(f"\n📝 Content:")
        content_preview = result['content'][:400] + "..." if len(result['content']) > 400 else result['content']
        print(content_preview)
    
    print(f"\n{'='*80}\n")

In [10]:
# Test with sample queries
test_queries = [
    "How can I request time off?",
]
for query in test_queries:
    test_retrieval(db, query, n_results=2)


🔍 QUERY: How can I request time off?


────────────────────────────────────────────────────────────────────────────────
RESULT #1 (distance: 0.9861)
────────────────────────────────────────────────────────────────────────────────
📄 Document: Gestione Ferie, Permessi e Assenze (POL-HR-001)
📍 Chunk: 5/13
📑 Section: 2.2 Pianificazione e Richiesta

🔍 Context:
Documento: Gestione Ferie, Permessi e Assenze (POL-HR-001), sezione 2.2 Pianificazione e Richiesta. Argomento: procedure di pianificazione e richiesta delle ferie con canale di invio tramite portale HR Zucchetti, requisiti di preavviso differenziati per la durata delle ferie e l'obbligo di presentare il piano ferie entro il 31 marzo per i periodi di picco estivi e natalizi. Contesto: parte della policy rivolta alla gestione delle assenze retribuite e non, mirata a garantire riposo dei dipendenti e continuità operativa tramite una gestione centralizzata delle richieste.

📝 Content:
Documento: Gestione Ferie, Permessi e Assenze (POL-HR